In [112]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%pip install mglearn
import mglearn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [113]:
# Load the training data
df_train = pd.read_csv('Train.csv')
# Load the test data
df_test = pd.read_csv('Test.csv')

In [114]:
df_train.head()

,w,Age,Accessibility,EdLevel,Employment,Gender,MentalHealth,MainBranch,YearsCode,YearsCodePro,Country,PreviousSalary,HaveWorkedWith,ComputerSkills,Employed
0,35381,>35,Yes,Other,1,Man,Yes,NotDev,35,15,United Kingdom of Great Britain and Northern I...,77556.0,C;C++;Go;PowerShell;Python;Ansible;Chef;Docker...,16,0
1,1453,>35,No,PhD,0,Man,No,Dev,35,30,United Kingdom of Great Britain and Northern I...,124092.0,C#;HTML/CSS;LISP;ASP.NET Core ;jQuery;Microsof...,7,1
2,24572,<35,No,Master,1,NonBinary,Yes,Dev,18,2,Denmark,78672.0,Bash/Shell;HTML/CSS;Java;JavaScript;Python;SQL...,12,1
3,42014,<35,No,Undergraduate,1,Man,Yes,Dev,10,1,France,44790.0,Rust;Docker;Kubernetes;AWS;Google Cloud;MongoD...,7,0
4,26711,>35,No,Master,0,Man,No,Dev,21,17,United Kingdom of Great Britain and Northern I...,155112.0,C#;HTML/CSS;JavaScript;Node.js;SQL;TypeScript;...,16,1


In [115]:
df_train.drop(['w'], axis=1, inplace=True)

In [116]:
#What correlates the most with "employed" column?
df.corr()['Employed'].sort_values()

C:\Users\Simon\AppData\Local\Temp\ipykernel_14040\1276420888.py:2: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.corr()['Employed'].sort_values()


Employment       -0.010255
YearsCode        -0.008982
YearsCodePro     -0.001812
PreviousSalary    0.002171
ComputerSkills    0.586196
Employed          1.000000
Name: Employed, dtype: float64

# Random forest

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [ ]:
# Step 1: Load your data
train_df = pd.read_csv('Train.csv')
test_df = pd.read_csv('Test.csv')

In [ ]:
# Step 2: Identify categorical columns that need to be encoded
# You'll need to fill this list with the actual categorical columns of your dataset
categorical_cols = ['Age', 'EdLevel', 'Gender', 'MainBranch']
numerical_cols = ['YearsCode', 'YearsCodePro', 'PreviousSalary', 'ComputerSkills']

In [ ]:
# Step 3: Create a ColumnTransformer to transform the categorical columns
preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numerical_cols),
        ('cat', OneHotEncoder(), categorical_cols)
    ])

In [ ]:
# Step 4: Create a pipeline that includes the preprocessor and the Random Forest model
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
])

In [ ]:
# Step 5: Separate features and target variable
X_train = train_df.drop('Employed', axis=1)  
y_train = train_df['Employed']
X_test = test_df.drop('Employed', axis=1)
y_test = test_df['Employed']

In [ ]:
# Step 6: Train the pipeline on the training data
pipeline.fit(X_train, y_train)

# Step 7: Make predictions on the test data
y_pred = pipeline.predict(X_test)

In [117]:
# Step 8: Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='binary')  # Adjust based on your target variable
precision = precision_score(y_test, y_pred, average='binary')
recall = recall_score(y_test, y_pred, average='binary')

# Print the evaluation metrics
print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")

Accuracy: 0.7582921185171742
F1 Score: 0.7779584010670668
Precision: 0.764980736125912
Recall: 0.7913839891451832


In [118]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

"""
# Define the hyperparameters and their possible values
param_grid = {
    'classifier__n_estimators': [50, 100, 200],
    'classifier__max_depth': [None, 10, 20],
    'classifier__min_samples_split': [2, 5, 10],
    'classifier__min_samples_leaf': [1, 2, 4]
}

# Create a GridSearchCV object
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='f1', n_jobs=-1)

# Fit the grid search to your data
grid_search.fit(X_train, y_train)

# Get the best parameters and model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_"""

"\n# Create a pipeline with the preprocessor and the classifier\npipeline = Pipeline(steps=[\n    ('preprocessor', preprocessor),\n    ('classifier', RandomForestClassifier(random_state=42))\n])\n\n# Define the hyperparameters and their possible values\n# Notice that we prefix classifier__ to each classifier parameter\nparam_grid = {\n    'classifier__n_estimators': [50, 100, 200],\n    'classifier__max_depth': [None, 10, 20],\n    'classifier__min_samples_split': [2, 5, 10],\n    'classifier__min_samples_leaf': [1, 2, 4]\n}\n\n# Create a GridSearchCV object\ngrid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='f1', n_jobs=-1)\n\n# Fit the grid search to your data\ngrid_search.fit(X_train, y_train)\n\n# Get the best parameters and model\nbest_params = grid_search.best_params_\nbest_model = grid_search.best_estimator_"

In [119]:
"""#Print the best parameters and model
print(f"Best Parameters: {best_params}")
print(f"Best Model: {best_model}")"""

'#Print the best parameters and model\nprint(f"Best Parameters: {best_params}")\nprint(f"Best Model: {best_model}")'

Output: Best Parameters: {'classifier__max_depth': 10, 'classifier__min_samples_leaf': 4, 'classifier__min_samples_split': 10, 'classifier__n_estimators': 100}
Best Model: Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', 'passthrough',
                                                  ['YearsCode', 'YearsCodePro',
                                                   'PreviousSalary',
                                                   'ComputerSkills']),
                                                 ('cat', OneHotEncoder(),
                                                  ['Age', 'EdLevel', 'Gender',
                                                   'MainBranch'])])),
                ('classifier',
                 RandomForestClassifier(max_depth=10, min_samples_leaf=4,
                                        min_samples_split=10,
                                        random_state=42))])

### Tuned model

In [121]:
# Step 4: Create a pipeline that includes the preprocessor and the Random Forest model
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(max_depth=10, min_samples_leaf=4, min_samples_split=10, n_estimators=100, random_state=42))
])

# Step 6: Train the pipeline on the training data
pipeline.fit(X_train, y_train)

# Step 7: Make predictions on the test data
y_pred = pipeline.predict(X_test)

# Step 8: Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='binary') 
precision = precision_score(y_test, y_pred, average='binary')
recall = recall_score(y_test, y_pred, average='binary')

# Print the evaluation metrics
print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")

Accuracy: 0.7832478787603794
F1 Score: 0.8018746630168804
Precision: 0.7847227859404172
Recall: 0.8197930800542741


#### Further tuned model

In [122]:
# Define the hyperparameters and their possible values
param_grid = {
    'classifier__n_estimators': [85, 100, 115],
    'classifier__max_depth': [7, 10, 13],
    'classifier__min_samples_split': [10, 12, 15],
    'classifier__min_samples_leaf': [4, 6, 8]
}

# Create a GridSearchCV object
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='f1', n_jobs=-1)

# Fit the grid search to your data
grid_search.fit(X_train, y_train)

# Get the best parameters and model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

In [123]:
#Print the best parameters and model
print(f"Best Parameters: {best_params}")
print(f"Best Model: {best_model}")

Best Parameters: {'classifier__max_depth': 13, 'classifier__min_samples_leaf': 6, 'classifier__min_samples_split': 15, 'classifier__n_estimators': 115}
Best Model: Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', 'passthrough',
                                                  ['YearsCode', 'YearsCodePro',
                                                   'PreviousSalary',
                                                   'ComputerSkills']),
                                                 ('cat', OneHotEncoder(),
                                                  ['Age', 'EdLevel', 'Gender',
                                                   'MainBranch'])])),
                ('classifier',
                 RandomForestClassifier(max_depth=13, min_samples_leaf=6,
                                        min_samples_split=15, n_estimators=115,
                                        random_state=42))])


In [124]:
# Step 4: Create a pipeline that includes the preprocessor and the Random Forest model
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(max_depth=13, min_samples_leaf=6, min_samples_split=15, n_estimators=115, random_state=42))
])

# Step 6: Train the pipeline on the training data
pipeline.fit(X_train, y_train)

# Step 7: Make predictions on the test data
y_pred = pipeline.predict(X_test)

# Step 8: Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='binary') 
precision = precision_score(y_test, y_pred, average='binary')
recall = recall_score(y_test, y_pred, average='binary')

# Print the evaluation metrics
print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")

Accuracy: 0.7846091020463724
F1 Score: 0.8040615841829364
Precision: 0.7832730197024528
Recall: 0.8259837177747625
